In [32]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import time
from sodapy import Socrata

In [27]:
#json file
crime_df = pd.DataFrame()
for i in range(500):
    offset = 2000*i
    limit = 2000
    df = pd.read_json(f"https://data.cityofchicago.org/resource/crimes.json?$limit=2000&$offset={offset}")
    crime_df = crime_df.append(other=df)

In [33]:
#selection of data from main table
new_crime_df = crime_df[['ward','primary_type','arrest','location_description','year','latitude','longitude']].copy()
new_crime_df.head()

,ward,primary_type,arrest,location_description,year,latitude,longitude
0,16.0,THEFT,False,SIDEWALK,2019,41.771737,-87.664006
1,28.0,NARCOTICS,True,STREET,2019,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,POLICE FACILITY/VEH PARKING LOT,2019,41.873699,-87.704705
3,24.0,BATTERY,True,STREET,2019,41.865495,-87.728818
4,17.0,ROBBERY,False,GAS STATION,2019,41.771532,-87.702965


In [57]:
#transform dataframe
transf_crime_df = ['ward','primary_type','arrest','year','location_description','latitude','longitude']
transf_crime_chicago = new_crime_df[transf_crime_df].copy()

In [58]:
# Rename the column headers
transf_crime_chicago = transf_crime_chicago.rename(columns={"ward": "ward_no",
                                                          "primary_type": "Crimetype",
                                                          "arrest": "Arrest",
                                                           "year":"Year",
                                                            "location_description":"Crime_location",
                                                            "latitude":"Latitude",
                                                            "longitude":"Longitude"})

In [59]:
transf_crime_chicago.head()

,ward_no,Crimetype,Arrest,Year,Crime_location,Latitude,Longitude
0,16.0,THEFT,False,2019,SIDEWALK,41.771737,-87.664006
1,28.0,NARCOTICS,True,2019,STREET,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,2019,POLICE FACILITY/VEH PARKING LOT,41.873699,-87.704705
3,24.0,BATTERY,True,2019,STREET,41.865495,-87.728818
4,17.0,ROBBERY,False,2019,GAS STATION,41.771532,-87.702965


In [60]:
transf_crime_chicago.shape


(1000000, 7)

In [4]:
#connection with database name "chicago_crime_db"
rds_connection_string = f"postgres:sherkhan@localhost:5432/chicago_crime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [66]:
#inserting and creating table in database
transf_crime_chicago.to_sql(name='chicago_crimes', con=engine, if_exists='append', index=False)

In [34]:
#reading data from database
df = pd.read_sql_query('select * from chicago_crimes', con=engine)


In [35]:
#changing name of coulms from previous table 
df.columns = ['ward_no', 'crime_type', 'arrest', 'year', 'crime_location', 'latitude',
       'longitude']

In [36]:
df.shape

(1000000, 7)

In [25]:
df_crime = df
df_crime.head()

,ward_no,crime_type,arrest,year,crime_location,latitude,longitude
0,16.0,THEFT,False,2019,SIDEWALK,41.771737,-87.664006
1,28.0,NARCOTICS,True,2019,STREET,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,2019,POLICE FACILITY/VEH PARKING LOT,41.873699,-87.704705
3,24.0,BATTERY,True,2019,STREET,41.865495,-87.728818
4,17.0,ROBBERY,False,2019,GAS STATION,41.771532,-87.702965


In [26]:
#connection with database name "chicago_crime_db"
rds_connection_string = f"postgres:sherkhan@localhost:5432/chicago_crime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
#inserting and creating table in database with table name chicago_crime
df_crime.to_sql(name='chicago_crime', con=engine, if_exists='append', index=False)


In [37]:
#retriving data from database
pd.read_sql_query('select * from chicago_crime', con=engine).head()

,ward_no,crime_type,arrest,year,crime_location,latitude,longitude
0,16.0,THEFT,False,2019,SIDEWALK,41.771737,-87.664006
1,28.0,NARCOTICS,True,2019,STREET,41.879612,-87.709447
2,24.0,INTERFERENCE WITH PUBLIC OFFICER,True,2019,POLICE FACILITY/VEH PARKING LOT,41.873699,-87.704705
3,24.0,BATTERY,True,2019,STREET,41.865495,-87.728818
4,17.0,ROBBERY,False,2019,GAS STATION,41.771532,-87.702965


In [ ]:
#By using Chicago Crime API we retrive data and save in database "chicago_crime_db" table name "chciago_crime"
# we export the cvs file from database chciago_crime
# will use that table 